In [15]:
import os 
os.makedirs ("test1")

In [17]:
%%writefile test1/test_1.py
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
num_workers=os.cpu_count()

def create_dataloaders (
                            train_dir:str,
                            test_dir:str,
                                transform:transforms.Compose,
                                batch_size:int,
                                num_workers:int=num_workers):
    # Use ImageFolder to create datasets
    train_data=datasets.ImageFolder(train_dir, transform=transform)
    test_data=datasets.ImageFolder(test_dir,transform=transform)
    #Get class names 
    class_names=train_data.classes
    # Turn datasets into dataloaders
    train_dataloader=DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers,pin_memory=True    )
    test_dataloader=DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
return train_dataloader, test_dataloader, class_names


Overwriting test1/test_1.py


In [18]:
%%writefile test1/model_builder.py

import torch

from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/
  
  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )
    
  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing test1/model_builder.py


In [19]:
from test1 import model_builder